<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-файла-и-изучение-общей-информации" data-toc-modified-id="Загрузка-файла-и-изучение-общей-информации-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка файла и изучение общей информации</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответы-на-вопросы" data-toc-modified-id="Ответы-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответы на вопросы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Загрузка файла и изучение общей информации

In [1]:
# импорт табличного редактора Pandas и чтение данных
import pandas as pd

df = pd.read_csv('data.csv')

Описание данных:
* `children` — количество детей в семье  
* `days_employed` — общий трудовой стаж в днях  
* `dob_years` — возраст клиента в годах  
* `education` — уровень образования клиента  
* `education_id` — идентификатор уровня образования  
* `family_status` — семейное положение  
* `family_status_id` — идентификатор семейного положения  
* `gender` — пол клиента  
* `income_type` — тип занятости  
* `debt` — имел ли задолженность по возврату кредитов  
* `total_income` — ежемесячный доход  
* `purpose` — цель получения кредита  

In [2]:
# предварительный просмотр данных
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
# проверка корректности записи наименований столбцов таблицы
# (латиница и строчные буквы / отсутствие пробелов / разделение слов нижним подчеркиванием)

df.columns # требования соблюдены

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [4]:
# общая информация о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В представленной таблице содержатся различные типы данных (целые и вещественные числа / строки). Для некоторых столбцов (например, `days_employed`) количество строк меньше их общего числа в таблице – что говорит о наличии пропусков данных.

In [5]:
# количественные характеристики числовых столбцов (int64 и float64)
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


На примере столбца `days_employed` (т.е. общий трудовой стаж в днях) можно увидеть, что присутствуют отрицательные числа, а также чрезмерно большие числа ($401755\div250\approx1607$ лет стажа, где 250 – количество рабочих дней в году). 

**Вывод**

Для реализации цели исследования (см. ниже), ввиду выявленных проблем в данных (см. выше), требуется выполнить предобработку данных. Исходные данные предоставлены в «сыром» виде.

Цель исследования состоит в том, чтобы ответить на представленные ниже вопросы:
* есть ли зависимость между наличием детей и возвратом кредита в срок?  
* есть ли зависимость между семейным положением и возвратом кредита в срок?  
* есть ли зависимость между уровнем дохода и возвратом кредита в срок?  
* как разные цели кредита влияют на его возврат в срок?

## Предобработка данных

### Обработка пропусков

In [6]:
# проверка столбцов на наличие пропусков данных
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски выявлены в столбцах `days_employed` и `total_income`. Поскольку столбец `days_employed` среди своих значений содержит отрицательные и чрезмерно большие значения, требуется дополнительная обработка.

In [7]:
# для столбцов 'days_employed' и 'total_income' необходимо откорректировать отрицательные и чрезмерно большие значения

# корректировка отрицательных значений – замена на положительные
df['children'] = df['children'].apply(abs) # результат можно проверить командой df.describe()
df['days_employed'] = df['days_employed'].apply(abs)

In [8]:
# корректировка чрезмерно больших значений
# для колонки 'children' примем максимальное число детей за приятное и допустимое исключение

# для колонки days_employed откорректируем чрезмерно большие значения до ближайшего наибольшего (в учебных целях)
# мужчины – 42 года или приблизительно 10500 дней, женщины – 37 лет или приблизительно 9250 дней, для остальных 'XNA' – 38 лет или приблизительно 9500 дней.
# группа XNA выявлена при помощи df['gender'].value_counts()

def work_years(row):
    """
    Возвращает откорректированный рабочий стаж в днях для чрезмерно больших значений используя правила:
    - мужчина "М", табличное значение превышает 10500 дней
    - женщина "F", табличное значение превышает 9250 дней
    - неустановленная категория "XNA", табличное значение превышает 9500 дней
    """

    total_days = row['days_employed']
    gender_status = row['gender']
    
    if (gender_status == 'M') & (total_days > 10500):
        return 10500
    elif (gender_status == 'F') & (total_days > 9250):
        return 9250
    elif (gender_status == 'XNA') & (total_days > 9500):
        return 9500
    else:
        return total_days
        

df['days_employed'] = df.apply(work_years, axis=1)

In [9]:
# возвращаемся к заполнению пропусков
# для столбца 'days_employed' – укажем срднее
df['days_employed'].fillna(df['days_employed'].mean(), inplace=True)

In [10]:
# столбец 'total_income'
# распределение 'income_type' при пропусках в 'total_income'
df[df['total_income'].isnull()]['income_type'].value_counts(normalize=True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

In [11]:
# распределение 'income_type' при исключении пропусков из 'total_income'
df[~df['total_income'].isnull()]['income_type'].value_counts(normalize=True)

сотрудник          0.517493
компаньон          0.236525
пенсионер          0.177924
госслужащий        0.067800
безработный        0.000103
предприниматель    0.000052
студент            0.000052
в декрете          0.000052
Name: income_type, dtype: float64

Первые три значимые категории (т.е. доля исчисляется десятками процентов – `сотрудник`, `компаньон` и `пенсионер`) практически повторяют друг друга. Это дает основания предположить, что некоторая связь между типом занятости `income_type` и наличием пропусков в столбце с уровнем дохода `total_income` существует.

In [12]:
# создадим словарь из медианных значений для каждой группы
df.groupby('income_type')['total_income'].median().to_dict()

{'безработный': 131339.7516762103,
 'в декрете': 53829.13072905995,
 'госслужащий': 150447.9352830068,
 'компаньон': 172357.95096577113,
 'пенсионер': 118514.48641164352,
 'предприниматель': 499163.1449470857,
 'сотрудник': 142594.39684740017,
 'студент': 98201.62531401133}

In [13]:
# для столбца 'days_employed' применим следующий подход по корректировки пропусков
def total_income_imputation(row):
    """
    Функция заполняет пропуски (отсутствующие данные) используя правила:
    - если в ячейке столбца 'total_income' фиксируется пропуск, то его заполняет медиана взятая с учетом категории 'income_type'
    - перечень категорий и соответствующие медианы отражены ниже в словаре 'median_dict_by_group'
    """

    income_type = row['income_type']
    total_income = row['total_income']
   
    median_dict_by_group = {
         'безработный': 131339,
         'в декрете': 53829,
         'госслужащий': 150447,
         'компаньон': 172357,
         'пенсионер': 118514,
         'предприниматель': 499163,
         'сотрудник': 142594,
         'студент': 98201
    }
    
 
    if total_income != total_income: # проверка, является ли значение в ячейке 'NaN'
        return median_dict_by_group[income_type] # если это 'NaN', заполняем пропуск медианой по соответствующей группе
    else:
        return total_income
        

df['total_income'] = df.apply(total_income_imputation, axis=1)

In [14]:
# проведем повторную проверку на наличие пропусков
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [15]:
# проверка столбца 'dob_years'
df['dob_years'].value_counts().sort_index().head(5)

0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64

Выявлено, что для заёмщиков в количестве 101 человека указан возраст 0 лет.

In [16]:
# заменим возраст 0 лет на медианное значение по столбцу
df['dob_years']=df['dob_years'].replace(0, df['dob_years'].median())

In [17]:
# повторная проверка столбца 'dob_years'
df['dob_years'].value_counts().sort_index().head(5)

19     14
20     51
21    111
22    183
23    254
Name: dob_years, dtype: int64

In [18]:
# проверка столбца 'gender'
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [19]:
# удаляем строку, которая содержит пол 'XNA'
df = df[df['gender'] !='XNA']

In [20]:
# повторная проверка столбца 'gender'
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

**Вывод**

В ходе выполнения предобработки было выявлено отсутствие данных в некоторых колонках, а также их несоответствие приемлемым величинам. Были предприняты действия, которые позволили заменить отсутствующие значения и усреднить (медианой и средним) чрезмерно большие величины. Из таблицы была полностью удалена одна строка (пол 'XNA'), т.к. её отсутствие вряд ли повлияет на конечный результат.

### Замена типа данных

In [21]:
# заменим тип данных в колонках 'days_employed' и 'total_income' с вещественного (float64) на целочисленный (int64) в учебных целях
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [22]:
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')

In [23]:
# проверка перевода данных 'days_employed' и 'total_income' в нужный тип - целочисленный (int64)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

С данными в столбцах `days_employed` и `total_income` можно оперировать как с целочисленными значениями.

### Обработка дубликатов

In [24]:
# разное написание одинаковых слов фиксируется в колонке 'education'
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [25]:
# приводим написание к нижнему регистру
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower() # в столбце дубликаты отсутствуют, но некоторые слова написаны с заглавной буквы

In [26]:
# повторы отсутствуют
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [27]:
# проверяем общее количество дубликатов во всей таблице
df.duplicated().sum()

72

In [28]:
# удаляем выявленные дубликаты
df = df.drop_duplicates().reset_index(drop=True)

In [29]:
# повторно проверяем общее количество дубликатов во всей таблице
df.duplicated().sum()

0

**Вывод**

Дубликаты были выявлены и удалены.

### Лемматизация

In [30]:
# цели заема совпадают по смыслу, но несколько различаются по описанию – например, «операции с недвижимостью» и «операции с жильем»
df['purpose'].value_counts().head(10)

свадьба                                  791
на проведение свадьбы                    767
сыграть свадьбу                          765
операции с недвижимостью                 675
покупка коммерческой недвижимости        661
операции с жильем                        652
покупка жилья для сдачи                  651
операции с коммерческой недвижимостью    650
покупка жилья                            646
жилье                                    646
Name: purpose, dtype: int64

In [31]:
# импортируем библиотеку для лемматизации, т.е. для приведения слова к его словарной форме (лемме).
from pymystem3 import Mystem
m = Mystem()

In [32]:
def lemmas_extractor(row):
    """
    Возвращает базовое слово используя правила:
    - если в ячейке 'purpose' содержится слово 'жилье' (из списка 'possible_options'), возвращается базовое слово 'недвижимость'
    - если содержится любое другое слово, возвращается это слово в базовой форме
    """
    possible_options = ['жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба'] # базовые слова, которые присутствуют во всех целевых тратах
    lemmas = m.lemmatize(row)
    
    if possible_options[0] in lemmas:
        return 'недвижимость' # сводим 'жилье' в категорию – 'недвижимость'
    else:
        return set(possible_options[1:]).intersection(set(lemmas)).pop() # извлекаем оставшиеся базовые слова через пересечение словарей


df['purpose_clean'] = df['purpose'].apply(lemmas_extractor)

In [33]:
# в дополнительно созданной колонке 'purpose_clean' теперь присутствуют только базовые слова
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_clean
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,9250,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Лемматизация позволила выделить базовые слова из общего описания, что в свою очередь помогло сузить и конретизировать цели для получения кредита.

### Категоризация данных

In [34]:
# определим столбцы, для которых можно выполнить категоризацию
# это столбцы с типом данных 'object' и дублирующим столбцом '_id', т.е. 'education', 'family_status'
# также в учебных целях выполним классификацию по возрастным группам – столбец 'total_income'
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
children            21452 non-null int64
days_employed       21452 non-null int64
dob_years           21452 non-null int64
education           21452 non-null object
education_id        21452 non-null int64
family_status       21452 non-null object
family_status_id    21452 non-null int64
gender              21452 non-null object
income_type         21452 non-null object
debt                21452 non-null int64
total_income        21452 non-null int64
purpose             21452 non-null object
purpose_clean       21452 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


In [35]:
# обработка столбца 'education'
# создаем «словарь» из ключа и значения
rest_dict_education = df[['education_id', 'education']]
rest_dict_education.head() # фиксируются повторы

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее


In [36]:
# удаляем повторы и обновляем индексы
rest_dict_education = rest_dict_education.drop_duplicates().reset_index(drop=True)
rest_dict_education.head() # повторы удалены

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [37]:
# обработка столбца 'family_status' по аналогии с указанным выше подходом
rest_dict_family_status = df[['family_status_id', 'family_status']]
rest_dict_family_status = rest_dict_family_status.drop_duplicates().reset_index(drop=True)
rest_dict_family_status.head()

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [38]:
# обработка столбца 'total_income'
# разделим уровень дохода на четыре (4) последовательные приблизительно равные по количеству условные группы – 'низкий', 'ниже среднего', 'выше среднего' и 'высокий'

labels = ['низкий', 'ниже среднего', 'выше среднего', 'высокий']

df['total_income_group'] = pd.qcut(df['total_income'], 4, labels=labels)

df['total_income_group'].value_counts()

ниже среднего    5479
высокий          5363
низкий           5363
выше среднего    5247
Name: total_income_group, dtype: int64

**Вывод**

Применение категоризации позволяет получать из большого массива данных, представленных как в строковом, так и в числовом виде, компактное представление информации. Такой подход упрощает восприятие данных и делает их дальнейшую обработку менее ресурсозатратной.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
# построим сводную таблицу
data_pivot_debt = df.pivot_table(index=[df['children']],
                                 columns='debt',
                                 values='children',
                                 aggfunc='count')


# вывод данных в процентах, %
data_pivot_debt_percent = data_pivot_debt.div(data_pivot_debt.sum(axis=1),
                                              axis=0).applymap(lambda x: "{0:.1f}%".format(100*x))

# объединим обе таблицы
data_pivot_debt.merge(data_pivot_debt_percent, left_on='children', right_on='children', suffixes=('', ' %'))

debt,0,1,0 %,1 %
children,,,,
0,13026.0,1063.0,92.5%,7.5%
1,4410.0,445.0,90.8%,9.2%
2,1858.0,194.0,90.5%,9.5%
3,303.0,27.0,91.8%,8.2%
4,37.0,4.0,90.2%,9.8%
5,9.0,NaN,100.0%,nan%
20,68.0,8.0,89.5%,10.5%


**Вывод**

Установлена зависимость между наличием детей и возвратом кредита в срок. Заемщиков без детей или с детьми и с непогашенным кредитом (1) кратно (приблизительно в 12 раз) меньше тех, кто не имеет обязательств перед банком (0). Преобладающее большинство заемщиков соблюдают финансовую дисциплину.  
**Примечание:** заемщик(-и) с количеством детей 5 и непогашенным кредитом (1) в исходных данных отсутствует.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
# построим сводную таблицу
data_pivot_family_status = df.pivot_table(index=[df['family_status']],
                                 columns='debt',
                                 values='family_status',
                                 aggfunc='count')

# вывод данных в процентах, %
data_pivot_family_status_percent = data_pivot_family_status.div(data_pivot_family_status.sum(axis=1), 
                                                                axis=0).applymap(lambda x: "{0:.1f}%".format(100*x))

# объединим обе таблицы
data_pivot_family_status.merge(data_pivot_family_status_percent,
                               left_on='family_status',
                               right_on='family_status',
                               suffixes=('', ' %'))

debt,0,1,0 %,1 %
family_status,,,,
в разводе,1110,85,92.9%,7.1%
вдовец / вдова,896,63,93.4%,6.6%
гражданский брак,3761,388,90.6%,9.4%
женат / замужем,11408,931,92.5%,7.5%
не женат / не замужем,2536,274,90.2%,9.8%


**Вывод**

Во всех категориях значительно преобладают люди без финансовых обязательств (0) перед банком.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
# построим сводную таблицу
data_pivot_total_income_group = df.pivot_table(index=[df['total_income_group']],
                                 columns='debt',
                                 values='total_income_group',
                                 aggfunc='count')

# вывод данных в процентах, %
data_pivot_total_income_group_percent = data_pivot_total_income_group.div(data_pivot_total_income_group.sum(axis=1),
                                                                          axis=0).applymap(lambda x: "{0:.1f}%".format(100*x))

# объединим обе таблицы
data_pivot_total_income_group.merge(data_pivot_total_income_group_percent,
                                    left_on='total_income_group',
                                    right_on='total_income_group',
                                    suffixes=('', ' %'))

debt,0,1,0 %,1 %
total_income_group,,,,
низкий,4936,427,92.0%,8.0%
ниже среднего,4996,483,91.2%,8.8%
выше среднего,4799,448,91.5%,8.5%
высокий,4980,383,92.9%,7.1%


**Вывод**

Заемщики распределены равномерно по уровням достатка. При этом большая их часть старается возвращать кредиты вовремя.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
# построим сводную таблицу
data_pivot_purpose_clean = df.pivot_table(index=[df['purpose_clean']],
                                 columns='debt',
                                 values='purpose_clean',
                                 aggfunc='count')

# вывод данных в процентах, %
data_pivot_purpose_clean_percent = data_pivot_purpose_clean.div(data_pivot_purpose_clean.sum(axis=1),
                                                                axis=0).applymap(lambda x: "{0:.1f}%".format(100*x))

# объединим обе таблицы
data_pivot_purpose_clean.merge(data_pivot_purpose_clean_percent,
                               left_on='purpose_clean',
                               right_on='purpose_clean',
                               suffixes=('', ' %'))

debt,0,1,0 %,1 %
purpose_clean,,,,
автомобиль,3903,403,90.6%,9.4%
недвижимость,10028,782,92.8%,7.2%
образование,3643,370,90.8%,9.2%
свадьба,2137,186,92.0%,8.0%


**Вывод**

Лидирующее место занимают кредиты на покупку недвижимости. Второе и третье место делят кредиты на приобретение автомобиля и получения образования соответственно. Во всех случаях количество заёмщиков с обязательствами перед банком (1) в меньшинстве.

## Общий вывод

В ходе выполненного анализа входных данных от банка со статистикой о платёжеспособности клиентов, была выявлена взаимосвязь между различными категориями клиентов и фактом возврата кредита заёмщиком.

С увеличением числа детей в семье, количество желающих взять кредит в банке резко сокращается. Охотнее всего пользуются кредитом люди без детей. Среди них наблюдается наибольший процент возврата 92.5% заема.

Меньше всего нарушителей финансовой дисциплины в категории вдовец / вдова, а именно 6.6%.

Наблюдается востребованность в заемных средствах всеми категориям.

Среди приоритетных целей кредита ожидаемо на первом месте стоит приобретением недвижимости.

В целом можно констатировать, что для всех рассмотренных случаев нарушители финансовой дисциплины составляют меньшинство – до 10.5%. По рассматриваемым категориям:
* заемщики без детей: количество лиц, вернувший кредит, превышает приблизительно в 12 раз нарушителей финансовой дисциплины (13026 / 1063)  
* семейные люди: количество лиц, вернувший кредит, превышает приблизительно в 12 раз нарушителей финансовой дисциплины  
* заемщики с низким уровнем дохода: количество лиц, вернувший кредит, превышает приблизительно в 11 раз нарушителей финансовой дисциплины  
* заемщики, взявшие кредит на приобретение недвижимости: количество лиц, вернувший кредит, превышает приблизительно в 13 раз нарушителей финансовой дисциплины  